In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

from datetime import date
import re
from datetime import datetime

/Users/Rong/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Entity Resolution Problem Statement

Journalists, academics, and businesses work hard to get big masses of data to learn about what people or organizations are doing. Unfortunately, once we get the data, we often can’t answer our questions because we can’t tell who is who.

In much real-world data, we do not have a way of absolutely deciding whether two records, say David Guy-Brizan and D Guy-Brizan are referring to the same person. If these were records of Professors at the University of San Francisco data, did a David Guy-Brizan give two lectures on two separate topics or did Desmond Guy-Brizan give the lecture on the second topic? Perhaps it could even be on completely separate topics.

People are pretty good at making these calls, if they have enough information. For example, I would be pretty confident that the following two records are the about the same person.

|first name | last name | Topic                   | hours   |
| --- | --- | --- |
|David      | Guy-Brizan   | Machine Learning   | 2pm - 4pm Thurs |
|D          | Guy-Brizan   | Algorithms |   2pm - 4pm Tuesdays|

If we have to decide which records in our data are about the same person or organization, then we could just go through by hand, compare every record, and decide which records are about the same entity.

This is very, very boring and can take a long time. 

# Exploring the data & Preprocessing

### We can do a combination of the following for data normalization: 
1. Convert all entities to upper or lower case, and remove whitespace
2. Run a spell checker to remove known typographical errors 
3. Replace nicknames, and expand abbreviations 
4. Perform looksups in lexicons 
5. Tokenize, Stem, or Lemmatize words 

### We can do a combination of the following for missing values: 
1. Set to Nan, Null, or remove 
2. Missing entries can also be filled by aggregating other fields or taking means 

In [2]:
# Read in the data
entities = pd.read_csv("/Users/Rong/Documents/USF/EntityResolution/Model1/named_resolution.csv")
articles = pd.read_json("/Users/Rong/Documents/USF/EntityResolution/Model1/articles.json")


In [3]:
# View data head

print(entities.head(5))
# select only relevant columns 
print(entities.head()[['article_id','name','type','paragraph']])

         id  article_id                               model         name      type  entity_id  paragraph  sentence
0  27303856  331842      english.all.3class.distsim.crf.ser  Broadway     LOCATION  2551       1          1       
1  27303857  331842      english.all.3class.distsim.crf.ser  Daniel Fish  PERSON    85567      1          2       
2  27303858  331842      english.all.3class.distsim.crf.ser  Rodgers      PERSON    70833      1          2       
3  27303859  331842      english.all.3class.distsim.crf.ser  Hammerstein  PERSON    98182      1          2       
4  27303860  331842      english.all.3class.distsim.crf.ser  Oklahoma     LOCATION  1332       1          2       
   article_id         name      type  paragraph
0  331842      Broadway     LOCATION  1        
1  331842      Daniel Fish  PERSON    1        
2  331842      Rodgers      PERSON    1        
3  331842      Hammerstein  PERSON    1        
4  331842      Oklahoma     LOCATION  1        


In [4]:
# print(articles.head(5)[['content', 'id']])

# Get all entities that are type PEOPLE only 
entity_people = entities[entities['type'] == 'PERSON']
print(entity_people.head(10)[['article_id','name','type','paragraph']])

# Capitalise all names and remove spaces and non alphabetical characters
entity_people_names_CAPITALS = entity_people['name'].str.upper().str.replace('\W+', '')
entity_people_names_CAPITALS.head(10)

    article_id                  name    type  paragraph
1   331842      Daniel Fish           PERSON  1        
2   331842      Rodgers               PERSON  1        
3   331842      Hammerstein           PERSON  1        
6   331842      Damon Daunno          PERSON  3        
9   331842      Fish                  PERSON  6        
11  331842      Oscar Hammerstein II  PERSON  7        
12  331842      Fish                  PERSON  8        
13  331842      Trevor Nunn           PERSON  8        
14  331842      Susan Stroman         PERSON  8        
17  331842      Lynn Riggs            PERSON  9        


1     DANIELFISH        
2     RODGERS           
3     HAMMERSTEIN       
6     DAMONDAUNNO       
9     FISH              
11    OSCARHAMMERSTEINII
12    FISH              
13    TREVORNUNN        
14    SUSANSTROMAN      
17    LYNNRIGGS         
Name: name, dtype: object

In [5]:
# check to see if we have duplicate Names
entity_people_names_CAPITALS.duplicated().head(20)  

## YES, WE DO!

1     False
2     False
3     False
6     False
9     False
11    False
12    True 
13    False
14    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
Name: name, dtype: bool

# Libraries for distributed representations of words 

There are several methods to compute the DRs of words: 
1. word2Vec https://github.com/maxoodf/word2vec
2. GloVe https://nlp.stanford.edu/projects/glove/
3. fastText https://fasttext.cc/




# Model 0 - Clustering entities with GloVe

#### GloVe = Global Vectors for Word Representation

GloVe is an unsupervised learning algorithm for obtaining vector representations for words. Training is performed on aggregated global word-word co-occurrence statistics from a corpus, and the resulting representations showcase interesting linear substructures of the word vector space.




#### Inputs: Entity 1, Context String Entity 1, Entity 2, Context String Entity 2 
#### Outputs: True/False 


#### Preprocessing 
Convert all Entities into lower or upper case format
Select only entities that are categorised as 'People'

#### Clustering
for each (Entity + Context String) in the dataset 
        if we find a match 
            add it to the Map 
        else 
            create new entry in the Map 
        
#### Post Processing 
Select the the entity representative of each bucket

    

# FuzzyWuzzy Playground
Fuzzy string matchung uses Levenshtein Distance to calculate the differences between sequences in a simple-to-use package.



In [6]:
# fuzz is used to compare TWO strings
from fuzzywuzzy import fuzz

# process is used to compare a string to MULTIPLE other strings
from fuzzywuzzy import process


In [7]:
# for each name, check for similar entries
def get_ratio(row): 
    name = fuzz.token_set_ratio(entity_people['name'], row['name'])
#    print("one", row['name'])
 #   print(name)
  #  print(entity_people['name'])
    entity_type = fuzz.token_set_ratio(entity_people['type'], row['type'])
    article_id = fuzz.token_set_ratio(entity_people['article_id'], row['article_id'])
    
#     weights = [8.0, 2.0]
        
#     return np.average(np.array([name, entity_type]), weights=weights)
    return np.average(np.array([name, entity_type, article_id]))

In [8]:
# add score column 
entity_people['SCORE'] = entities.apply(get_ratio, axis=1)

entity_people.head()

# What does this score tell us?


,id,article_id,model,name,type,entity_id,paragraph,sentence,SCORE
1,27303857,331842,english.all.3class.distsim.crf.ser,Daniel Fish,PERSON,85567,1,2,100.0
2,27303858,331842,english.all.3class.distsim.crf.ser,Rodgers,PERSON,70833,1,2,100.0
3,27303859,331842,english.all.3class.distsim.crf.ser,Hammerstein,PERSON,98182,1,2,100.0
6,27303863,331842,english.all.3class.distsim.crf.ser,Damon Daunno,PERSON,546335,3,1,100.0
9,27303870,331842,english.all.3class.distsim.crf.ser,Fish,PERSON,85567,6,2,100.0


In [9]:

# Convert all entities to Upper case and remove white space
entity_people = entity_people.assign(name=entities['name'].str.upper().str.replace('\W+', ''))

# Sort scores in descending order
entity_people.loc[entities['type'] == 'PERSON'].sort_values('SCORE',ascending=True).head(5)
# entity_people.loc[entities['type']=='PERSON'].sort_values('SCORE',ascending=False).head(70)


,id,article_id,model,name,type,entity_id,paragraph,sentence,SCORE
1089,45270731,423929,english.all.3class.distsim.crf.ser,DOUBT,PERSON,886957,12,1,35.0
1059,45269526,423928,english.all.3class.distsim.crf.ser,RIVAS,PERSON,886939,9,1,35.0
2035,45386277,424545,english.all.3class.distsim.crf.ser,ABRAMS,PERSON,888323,5,1,35.0
2042,45041895,423294,english.all.3class.distsim.crf.ser,OBAMA,PERSON,319,3,2,35.0
1022,45272338,423931,english.all.3class.distsim.crf.ser,RIVERA,PERSON,178524,7,1,35.0


In [10]:
process.extract("DONALDTRUMP", entity_people.name, scorer=fuzz.token_sort_ratio)

[('DONALDTRUMP', 100, 528),
 ('DONALDTRUMP', 100, 864),
 ('DONALDTRUMP', 100, 1112),
 ('DONALDTRUMP', 100, 2104),
 ('DONALDJTRUMP', 96, 1893)]

# Applying Dedupe

In [11]:
entities.head()

# Get all data related to people only
entity_people = entities[entities['type'] == 'PERSON']

# print(entity_people[entity_people.name == 'HAMMERSTEIN'])
entity_people.head()

,id,article_id,model,name,type,entity_id,paragraph,sentence
1,27303857,331842,english.all.3class.distsim.crf.ser,Daniel Fish,PERSON,85567,1,2
2,27303858,331842,english.all.3class.distsim.crf.ser,Rodgers,PERSON,70833,1,2
3,27303859,331842,english.all.3class.distsim.crf.ser,Hammerstein,PERSON,98182,1,2
6,27303863,331842,english.all.3class.distsim.crf.ser,Damon Daunno,PERSON,546335,3,1
9,27303870,331842,english.all.3class.distsim.crf.ser,Fish,PERSON,85567,6,2


In [12]:
print("Shape of entities : {}".format(entities.shape))
print("Shape of entities : {}".format(entity_people.shape))

# Here we see that out of the 2391 data entries that we have, 1222 of those are classified as a PERSON.
# For our first model, we will use the dedupe library on type=PERSON


Shape of entities : (2391, 8)
Shape of entities : (1222, 8)


In [13]:
import logging; logging.disable(level=logging.NOTSET)

# Entity Resolution with Dedupe.io

Dedupe is a library that uses machine learning to perform de-duplication and entity resolution quickly on structured data.

https://github.com/dedupeio/dedupe

Dedupe will help us remove duplicate entries from our people dataset of name entities, whilst taking into account the paragraph information which provides context to the named entities. 

Dedupe takes in human training data and comes up with the best rules for your dataset to quickly and automatically find similar records, even with very large databases.


### Variable Types
A variable definition describes the records that you want to match. It is a dictionary where the keys are the fields and the values are the field specification. For our example, we will use the following:

#### String Types
String types are compared using affine gap string distance.

#### Text Types
Fields containing long blocks of text e.g. product descriptions or article abstracts we use the Text type fields. These are compared using the cosine similarity metric.

This is a measurement of the amount of words that two documents have in common. This measure can be made more useful as the overlap of rare words counts more than the overlap of common words.

If provided a sequence of example fields (i.e. a corpus) then dedupe will learn these weights for you.

#### Name Types
A Name variable should be used for a field that contains American names, corporations and households. It uses the probablepeople package to split apart a name string into components like give name, surname, generational suffix, for people names, and abbreviation, company type, and legal form for corporations.

### How it works 

Choose column types --> Train Model --> Review Clusters --> Add to clusters --> Polish Clusters

## Building blocks


#### Record similarity
We make the assumpiton that records that are more similar are more likely to be duplicates. 

The default way that this is done in Dedupe is to use what’s called a string metric. A string metric is a way of taking two strings and returning a number that is low if the strings are similar and high if they are dissimilar. 

There are lots of different string metrics, and we actually use a metric called the Affine Gap Distance. It counts the number of substitutions that must be made to turn one string into another. It is similar to our familiar Levenshtein distance, that is used in Fuzzy Wuzzy.


#### Regularized logistic regression
If we supply pairs of records that we label as either being duplicates or distinct, then Dedupe will learn a set of weights such that the record distance can easily be transformed into our best estimate of the probability that a pair of records are duplicates.

Once we have learned these good weights, we want to use them to find which records are duplicates. But turns out that doing this the naive way will usually not work, and we’ll have to do something smarter.


#### Active Learning
In order to learn those weights, Dedupe needs example pairs with labels. Most of the time, we will need to supply those labels.

But the whole point of Dedupe is to save people’s time, and that includes making good use of your labeling time so we use an approach called Active Learning.

To do this, we maintain a set of the pairs where there is disagreement: that is pairs which classifier believes are duplicates but which are not covered by the current blocking rules, and the pairs which the classifier believes are distinct but which are blocked together.

Dedupe picks, at random from this disagreement set, a pair of records and asks the user to decide. Once it gets this label, it relearns the weights and blocking rules. We then recalculate the disagreement set.






# Model 1 - Dedupe.io with name, article_id, and abs_paragraph

In [21]:
fields = [
{
"crf": True,
"type": "Name",
"field": "abs_name",
"log file": "/tmp/name.csv"
},
{
"type": "PositiveNumber",
"field": "abs_article_id"
},
{
"type": "PositiveNumber",
"field": "abs_paragraph"
}

]

# Our results for Model 1


In [190]:
deduped_people = pd.read_csv("/Users/Rong/Documents/USF/EntityResolution/Model1/DedupedPersons.csv")


grouped_clusterId_model1 = deduped_people.groupby('cluster_id')
# grouped_name = deduped_people.groupby('name')

trump = grouped_clusterId_model1.get_group('63758edf-0cdc-4c6e-bc20-ec1823f54dfb')

print(len(trump))
print(trump)

# grouped_name.get_group('Trump')

56
                               cluster_id        id  article_id                               model          name    type  entity_id  paragraph  sentence
373  63758edf-0cdc-4c6e-bc20-ec1823f54dfb  45273599  423932      english.all.3class.distsim.crf.ser  Trump         PERSON  167        5          2       
374  63758edf-0cdc-4c6e-bc20-ec1823f54dfb  45289264  424080      english.all.3class.distsim.crf.ser  Trump         PERSON  167        13         3       
375  63758edf-0cdc-4c6e-bc20-ec1823f54dfb  45289328  424080      english.all.3class.distsim.crf.ser  Trump         PERSON  167        27         3       
376  63758edf-0cdc-4c6e-bc20-ec1823f54dfb  45289377  424080      english.all.3class.distsim.crf.ser  Trump         PERSON  167        38         2       
377  63758edf-0cdc-4c6e-bc20-ec1823f54dfb  31202403  320418      english.all.3class.distsim.crf.ser  Trump         PERSON  167        4          1       
378  63758edf-0cdc-4c6e-bc20-ec1823f54dfb  45345139  424388      english.

In [191]:
#Find the total number of names relative to number of clusters
# print(deduped_people)
all_names = deduped_people['name']

#Find names
names_dict = {}

total = 0
for name in all_names:
        if name not in names_dict:
            names_dict[name] = total
            total = total + 1        
    
# for key, value in names_dict.items() :
#     print (key, value)

clusters = deduped_people['cluster_id']



# Find clusters
cluster_dict = {}

total = 0
for name in clusters:
        if name not in cluster_dict:
            cluster_dict[name] = total
            total = total + 1        
    
# for key, value in cluster_dict.items() :
#     print (key, value)

# print("Number of names", len(names_dict))
# print("Number of clusters", len(cluster_dict))
    

## Model 1 Overall data: 

#### Number of names: 452

#### Number of clusters: 294


In [133]:
def clusterIdToCount( dataframe ):
    mappedItems = {}
    for index, row in dataframe.iterrows():
        clusterId = row['cluster_id']
        if clusterId not in mappedItems:
            mappedItems[clusterId] = 1
        else:
            mappedItems[clusterId] += 1
            
    # for key, value in clusterToCount_Model1.items():
#     print (key, value)

    result = pd.DataFrame(list(mappedItems.items()))
    
    return result


In [310]:
# Create a dictionary that maps ClusterId's to number of occurences
# print(deduped_people.head(10)[['cluster_id','name']])

clusterIdWithName = deduped_people[['cluster_id','name']]
print(clusterIdWithName)
print(type(clusterIdWithName))

clusterToCount_Model1 = clusterIdToCount(clusterIdWithName)

clusterToCount_Model1.head(10)

# a scatter plot comparing
# deduped_people.plot(kind='scatter',x='name',y='cluster_id',color='red')
# plt.show()

                               cluster_id                     name
0    0074c072-db24-469b-ae55-cd37b0637e14  Miriam Buether         
1    02146eca-2281-4d98-b7f7-3838e3eb08f7  John Durham            
2    02146eca-2281-4d98-b7f7-3838e3eb08f7  Connecticut John Durham
3    02146eca-2281-4d98-b7f7-3838e3eb08f7  Durham                 
4    02146eca-2281-4d98-b7f7-3838e3eb08f7  John Durham            
5    02146eca-2281-4d98-b7f7-3838e3eb08f7  John Durham            
6    031da3d0-179b-43be-a47f-d73dd6d2367f  Jackie                 
7    054341de-275b-4c30-b6a0-9d3c262bdbb7  Tucker Carlson         
8    054341de-275b-4c30-b6a0-9d3c262bdbb7  Tucker Carlson         
9    0547d4c7-8c90-477b-bbac-be43c60ad5fd  Holly Metcalf Kinyon   
10   0547d4c7-8c90-477b-bbac-be43c60ad5fd  Kinyon                 
11   066e8640-3995-4436-8a87-cbf3274dce3c  Ilhan Omar             
12   06840a8a-ba96-48d7-a8cc-0299498be89b  Ingram                 
13   068b4c90-8e50-4e4d-b510-42e9b54762e3  Stephanie Nolasco  

,0,1
0,0074c072-db24-469b-ae55-cd37b0637e14,1
1,02146eca-2281-4d98-b7f7-3838e3eb08f7,5
2,031da3d0-179b-43be-a47f-d73dd6d2367f,1
3,054341de-275b-4c30-b6a0-9d3c262bdbb7,2
4,0547d4c7-8c90-477b-bbac-be43c60ad5fd,2
5,066e8640-3995-4436-8a87-cbf3274dce3c,1
6,06840a8a-ba96-48d7-a8cc-0299498be89b,1
7,068b4c90-8e50-4e4d-b510-42e9b54762e3,1
8,06aa896b-ed2b-414e-a293-b5c54a02b6c3,3
9,06b17667-dc3b-4405-b7dc-60aecedbe850,16


# Model 2 - Dedupe.io with the associated Article as input

For our second model, we include the article content associated with the entity to give us greater context when applying Machine Learning.

In [145]:
#Process article data
articles.head(10)[['content','description','id']]
# print(articles.head())

# create dictionary

article_id = {}

count = 0
duplicate = 0
for index, row in articles.iterrows():
        if row['id'] not in article_id:
            article_id[row['id']] = row['content']
            count = count + 1
        else:
            duplicate = duplicate + 1
        

print(count)        
print(duplicate)
len(article_id)


100
0


100

In [146]:
entity_people.head()

id  article_id                               model          name    type  entity_id  paragraph  sentence                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [147]:
#Combine article data into main sample data

entity_people['content'] = ""
# print(article_id.keys())

for index, row in entity_people.iterrows():
    a_id = row['article_id']
#     print(row[index], article_id.get(a_id))
    entity_people.at[index, 'content'] = article_id.get(a_id)
#     row['content'][index] =     

# entity_people.head()

In [148]:
# entity_people.head()

In [149]:
entity_people.to_csv(r'/Users/Rong/Documents/USF/EntityResolution/Model1/people_articles.csv', index = False)

In [150]:
print(entity_people.shape)

(1222, 9)


In [151]:
people_articles = pd.read_csv("/Users/Rong/Documents/USF/EntityResolution/Model1/people_articles.csv")

people_articles.head()


id  article_id                               model          name    type  entity_id  paragraph  sentence                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

## Applying Dedupe

In [152]:
fields = [
{
"type": "Name",
"field": "name",
},
{
"type": "PositiveNumber",
"field": "article_id"
},
{
"type": "PositiveNumber",
"field": "paragraph"
},
{
"type": "PositiveNumber",
"field": "sentence"
},
{
"type": "Text",
"field": "content"
}
]

# Output results for Model 2 

In [153]:
deduped_people_with_articles = pd.read_csv("/Users/Rong/Documents/USF/EntityResolution/Model1/people_articles_results.csv")

deduped_people_with_articles.head()

,cluster_id,id,article_id,model,name,type,entity_id,paragraph,sentence,content
0,00285e55-b46b-4cf1-8cb6-5c4393cb34f5,45318247,424195,english.all.3class.distsim.crf.ser,Mrs Webster,PERSON,887499,12,1,"An autistic boy who had his leg amputated is battling for a secondary school place after ""falling through the cracks"" of education provision.\nDaniel Webster, 11, has missed school for two years due to illness and school refusal - also known as school phobia. \nTrina Webster said she had letters from mainstream and special schools saying they could not cater for his needs. \nDudley Council said it was committed to giving children ""the means to reach their full potential.""\nMrs Webster, of Halesowen, spoke to the BBC after seeing Newsnight's investigation into why 1,500 children with special educational needs and disabilities (Send) are without a school place in England.\nDaniel was six when doctors found he had a tumour in his hip and thigh, said Mrs Webster. \nHe was signed off from primary school in 2017 as medically unfit when he was given morphine and other drugs to manage the pain. \nIn 2018, he had to have his leg amputated. \n""He had the operation in March but he was doing so well by May we wanted to see if he was ready to go back to school, but it was too soon. He started 'school refusing',"" said Mrs Webster. \nSchool refusal is a recognised psychological condition in which children develop an inherent fear of attending school. Daniel also suffers with selective mutism, an anxiety disorder that affects the ability to speak and communicate.\nHe did not return to primary school but has been been receiving tuition from Cherry Tree Learning Centre, which helps children who have emotional and physical needs.\nNow 11, he is due to start secondary school in September, and Mrs Webster said she applied to several special needs schools.\n""Mainstream schools will be noisy and too much pressure for him,"" she said. \n""But they all turned us down. One said he would be isolated because there was no peer group for him. Another said his needs were too complex.""\nInstead, the Websters were told Daniel had a place at Earls High School in Dudley. Mrs Webster said the school had previously told her it did not have the facilities to cater for her son. \nThe family will now attend a tribunal in July at Dudley Council to try to find Daniel a suitable place at a special school. \n""I'm not sending him to a school where he's going to fail, where he's not going to cope,"" said Mrs Webster. \n""It's so frustrating. It's not just his physical needs, it's emotional too. We don't want him in a noisy, pressure-filled environment.""\nCouncillor Ruth Buttery, cabinet member for children and young people, said: ""Working in conjunction with health and education providers, we are determined to provide the best possible outcomes for pupils with Send or additional health needs and we have recently appointed two specialist education officers to support and advice parents and carers."""
1,00285e55-b46b-4cf1-8cb6-5c4393cb34f5,45318255,424195,english.all.3class.distsim.crf.ser,Mrs Webster,PERSON,887499,15,2,"An autistic boy who had his leg amputated is battling for a secondary school place after ""falling through the cracks"" of education provision.\nDaniel Webster, 11, has missed school for two years due to illness and school refusal - also known as school phobia. \nTrina Webster said she had letters from mainstream and special schools saying they could not cater for his needs. \nDudley Council said it was committed to giving children ""the means to reach their full potential.""\nMrs Webster, of Halesowen, spoke to the BBC after seeing Newsnight's investigation into why 1,500 children with special educational needs and disabilities (Send) are without a school place in England.\nDaniel was six when doctors found he had a tumour in his hip and thigh, said Mrs Webster. \nHe was signed off from primary school in 2017 as medically unfit when he was given mo

In [169]:
#Find the number of names in each cluster
clusterIdWithName_Model2 = deduped_people_with_articles[['cluster_id','name']]
# print(clusterIdWithName.head(10))
# print(type(clusterIdWithName))

clusterToCount_Model2 = clusterIdToCount(clusterIdWithName_Model2)

print(clusterToCount_Model2.head(5))


                                      0   1
0  00285e55-b46b-4cf1-8cb6-5c4393cb34f5  4 
1  01f2271f-ed0b-43cc-8868-2e925a734106  74
2  024f034a-bb3c-4b22-8b9d-25e5609785f2  1 
3  02983e86-efa0-47b7-97f9-ed0b4a42bda6  1 
4  03952352-84e9-4c76-bee5-2e31d3652a95  1 


In [172]:
#Find the total number of names relative to number of clusters
# print(deduped_people)
all_names = deduped_people_with_articles['name']

#Find names
names_dict = {}

total = 0
for name in all_names:
        if name not in names_dict:
            names_dict[name] = total
            total = total + 1        
    
# for key, value in names_dict.items() :
#     print (key, value)

clusters = deduped_people_with_articles['cluster_id']



# Find clusters
cluster_dict = {}

total = 0
for name in clusters:
        if name not in cluster_dict:
            cluster_dict[name] = total
            total = total + 1        
    
# for key, value in cluster_dict.items() :
#     print (key, value)

# print("Number of names", len(names_dict))
# print("Number of clusters", len(cluster_dict))
    

Number of names 448
Number of clusters 305


## Model 2 Overall data: 

#### Number of names: 448

#### Number of clusters: 305


In [206]:
grouped_clusterId_model2 = deduped_people_with_articles.groupby('cluster_id')
# grouped_name_model2 = deduped_people_with_articles.groupby('name')

# grouped_name_model2.get_group('Trump').head(5)
trump = grouped_clusterId_model2.get_group('01f2271f-ed0b-43cc-8868-2e925a734106')


# print(len(trump))
# trump.head(74)

# Compare Model 1 & Model 2

## True number of clusters = ?

In [379]:
def compareModels(name):
     
    print("Compare values for: ", name)
    
        
#Model 2
    #Get clusterId of input name
    grouped_name_model2 = deduped_people_with_articles.groupby('name')
#     print(grouped_name_model2.get_group(name)[['cluster_id', 'name']])
    clusterId2 = grouped_name_model2.get_group(name)[['cluster_id', 'name']]
    
    #pass the id into the next lines
    model2Id = clusterId2[0:1]
    print("Model 2 Id:", model2Id)

    grouped_clusterId_model2 = deduped_people_with_articles.groupby('cluster_id')
    model2 = grouped_clusterId_model2.get_group('01f2271f-ed0b-43cc-8868-2e925a734106')['name']
    size2 = len(model2)
#     print("Size of Model 2:", size2)
    
#Model 1
    #Get clusterId of input name
#     print(deduped_people[['cluster_id','name']])

    grouped_name_model1 = deduped_people.groupby('name')
    clusterId1 = grouped_name_model1.get_group(name)[['cluster_id', 'name']]
    
    #pass the id into the next lines    
    model1Id = clusterId1[0:1]
    print("Model 1 Id:", model1Id)

    grouped_clusterId_model1 = deduped_people.groupby('cluster_id')
    model1 = grouped_clusterId_model1.get_group('63758edf-0cdc-4c6e-bc20-ec1823f54dfb')['name']
    size1 = len(model1)
#     print("Size of Model 1:", size1)
    
    return model1, model2
    

In [382]:
name = 'Trump'

result1, result2 = compareModels(name)

print("Size of Model 1:", len(result1))
print("Size of Model 2:", len(result2))


print(result1)
print(result2)

Compare values for:  Trump
Model 2 Id:                              cluster_id   name
4  01f2271f-ed0b-43cc-8868-2e925a734106  Trump
Model 1 Id:                                cluster_id   name
373  63758edf-0cdc-4c6e-bc20-ec1823f54dfb  Trump
Size of Model 1: 56
Size of Model 2: 74
373    Trump       
374    Trump       
375    Trump       
376    Trump       
377    Trump       
378    Trump       
379    Trump       
380    Trump       
381    Trump       
382    Trump       
383    Trump       
384    Trump       
385    Trump       
386    Trump       
387    Trump       
388    Donald Trump
389    Trump       
390    Trump       
391    Trump       
392    Donald Trump
393    Trump       
394    Trump       
395    Trump       
396    Trump       
397    Trump       
398    Trump       
399    Trump       
400    Trump       
401    Trump       
402    Trump       
403    Donald Trump
404    Trump       
405    Trump       
406    Trump       
407    Trump       
408    Trump     